In [9]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'/home/ubuntu/varios/skforecast'

In [10]:
# Libraries
# ======================================================================================
import numpy as np
import pandas as pd
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import grid_search_forecaster
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression

In [11]:
# Data
# ======================================================================================
y = pd.Series(np.random.normal(size=50))
exog_1 = pd.Series([1]*25 + [2]*25, name='exog_1', dtype=float)
exog_2 = pd.Series([0]*25 + [1]*25, name='exog_2', dtype='category')
exog   = pd.concat((exog_1, exog_2), axis=1)

In [12]:
forecaster = ForecasterAutoreg(regressor=LGBMRegressor(), lags=5)

In [13]:
X_train, y_train = forecaster.create_train_X_y(
    y=y,
    exog=exog_2
)
print(X_train.dtypes)
X_train.head(3)

lag_1      float64
lag_2      float64
lag_3      float64
lag_4      float64
lag_5      float64
exog_2    category
dtype: object


,lag_1,lag_2,lag_3,lag_4,lag_5,exog_2
5,-0.782361,1.306765,2.052542,1.098706,-0.986584,0
6,-0.565109,-0.782361,1.306765,2.052542,1.098706,0
7,0.215025,-0.565109,-0.782361,1.306765,2.052542,0


In [14]:
forecaster.fit(y=y, exog=exog)
exog_predict = exog.copy()
exog_predict.index = pd.RangeIndex(50, 100)
forecaster.predict(steps=3, exog=exog_predict)

50    0.290106
51    0.640328
52    0.714016
Name: pred, dtype: float64

In [15]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from sklearn.preprocessing import OrdinalEncoder

categorical_cols = exog.select_dtypes(include="category").columns
ordinal_encoder = make_column_transformer(
    (
        OrdinalEncoder(dtype=int, handle_unknown="use_encoded_value", unknown_value=-1),
        make_column_selector(dtype_include="category"),
    ),
    remainder="passthrough",
    verbose_feature_names_out=False,
).set_output(transform="pandas")

exog_encoded = ordinal_encoder.fit_transform(exog)
exog_encoded[categorical_cols] = exog_encoded[categorical_cols].astype('category')

In [16]:
X_train, y_train = forecaster.create_train_X_y(
    y=y,
    exog=exog_encoded
)
print(X_train.dtypes)
X_train.head(3)

lag_1      float64
lag_2      float64
lag_3      float64
lag_4      float64
lag_5      float64
exog_2    category
exog_1     float64
dtype: object


,lag_1,lag_2,lag_3,lag_4,lag_5,exog_2,exog_1
5,-0.782361,1.306765,2.052542,1.098706,-0.986584,0,1.0
6,-0.565109,-0.782361,1.306765,2.052542,1.098706,0,1.0
7,0.215025,-0.565109,-0.782361,1.306765,2.052542,0,1.0


In [17]:
forecaster.fit(y=y, exog=exog_encoded)
exog_predict = exog_encoded.copy()
exog_predict.index = pd.RangeIndex(50, 100)
forecaster.predict(steps=3, exog=exog_predict)

50    0.290106
51    0.640328
52    0.714016
Name: pred, dtype: float64